In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import datetime
from requests.auth import HTTPBasicAuth

In [42]:
with open('./Toggl/private_info.json') as f:
    dic = json.load(f)

In [44]:
class toggl_connecer:
    def __init__(self,json_path):
        self._token = None
        self._mail_address = None
        self._workspace_id = None
            
        with open(json_path) as f:
            info_dic = json.load(f)
    
        self._token = info_dic["api-token"]
        self._mail_address = info_dic["mail_address"]
    
        # ワークスペースidの取得
        # get workspace id from api/v8/workspaces
        r = requests.get('https://www.toggl.com/api/v8/workspaces',
                         auth=(self._token, 'api_token'))
        
        data = r.json()
        Data = data[0]
        self._workspace_id = Data['id'] 
        
    def get_projects(self):
        # return projects dictionary
        p_dictionary = {}
        r = requests.get('https://www.toggl.com/api/v8/workspaces/{0}/projects'.format(self._workspace_id),
                         auth=(self._token, 'api_token'))

        data = r.json()
        for d in data:
            p_dictionary[d["name"]] = d["id"]
        self.projects = p_dictionary
        
        return p_dictionary
        
        
    def get_reports(self,start_date,end_date):
        """
        start_date:yyyy-mm-dd
        end_date:yyyy-mm-dd
        """
        params = {
            'user_agent': self._mail_address,
            'workspace_id': self._workspace_id,
            'since': start_date,
            'until': end_date,
        }
        r = requests.get('https://toggl.com/reports/api/v2/details',
                         auth=HTTPBasicAuth(self._token, 'api_token'),
                         params=params)
        json_r = r.json()
        
        # 必要なデータのみ抽出する
        use_cols = ['id','description','start','end','updated','dur','use_stop','project','task','tags']
        df = pd.DataFrame(json_r["data"],columns = use_cols)
       
        # 分データを追加
        df["minute"] = df["dur"]/(1000*60)
        df["minute"] = df["minute"].astype(int)
        return df

In [45]:
today = datetime.datetime.today()
last_year = datetime.date(year=today.year-1,month=today.month,day=today.day).strftime("%Y-%m-%d")
today = today.strftime("%Y-%m-%d")

In [46]:
last_year

'2020-04-14'

In [47]:
data = toggl_connecer('./Toggl/private_info.json').get_reports(last_year,today)

In [48]:
data

,id,description,start,end,updated,dur,use_stop,project,task,tags,minute
0,1964607665,物体検出入門,2021-04-12T22:19:03+09:00,2021-04-12T22:33:56+09:00,2021-04-12T22:33:57+09:00,893000,True,AI,None,[Learn_AI],14
1,1964525410,物体検出入門,2021-04-12T21:43:01+09:00,2021-04-12T21:56:51+09:00,2021-04-12T21:56:52+09:00,830000,True,AI,None,[Learn_AI],13
2,1964190596,物体検出入門,2021-04-12T18:31:15+09:00,2021-04-12T18:42:05+09:00,2021-04-12T18:47:41+09:00,650000,True,AI,None,[Learn_AI],10
3,1963233114,物体検出入門,2021-04-11T14:04:44+09:00,2021-04-11T14:10:27+09:00,2021-04-11T14:10:28+09:00,343000,True,AI,None,[Learn_AI],5
4,1962862995,食品ロス,2021-04-10T14:27:33+09:00,2021-04-10T15:06:12+09:00,2021-04-10T15:06:13+09:00,2319000,True,AI,None,[AI_Practice_compe],38
5,1962854500,食品ロス,2021-04-10T13:41:30+09:00,2021-04-10T13:47:03+09:00,2021-04-10T13:47:04+09:00,333000,True,AI,None,[AI_Practice_compe],5
6,1962839747,食品ロス,2021-04-10T12:22:39+09:00,2021-04-10T13:32:14+09:00,2021-04-10T13:41:20+09:00,4175000,True,AI,None,[AI_Practice_compe],69
7,1962835830,食品ロス,2021-04-10T12:03:41+09:00,2021-04-10T12:12:54+09:00,2021-04-10T12:12:55+09:00,553000,True,AI,None,[AI_Practice_compe],9
8,1962830736,SIGNATE_Quest,2021-04-10T11:38:52+09:00,2021-04-10T12:02:30+09:00,2021-04-10T12:02:32+09:00,1418000,True,AI,None,[食品ロス],23
9,1962828350,SIGNATE_Quest,2021-04-10T11:28:25+09:00,2021-04-10T11:32:24+09:00,2021-04-10T11:38:53+09:00,239000,True,AI,None,[食品ロス],3


## デザインメモ
- 概要
    - 直近12ヶ月
    - 縦軸（行側）にプロジェクト、横軸（列側）に日付の表を作る
- 詳細

In [49]:
class design_data():
    def __init__(self,data):
        self._df = None
        self._df = data
        self.today = datetime.date.today().strftime('%Y-%m-%d')
        
    def show_summary_views(self):
        tmp_df = self._df.copy()
        tmp_df["yyyymm"] = pd.to_datetime(tmp_df["start"]).dt.strftime("%Y-%m")
        agg_df = data[["yyyymm","project","minute"]].groupby(["yyyymm","project"]).sum()

In [57]:
data["yyyymm"] = pd.to_datetime(data["start"]).dt.strftime("%Y-%m")
data[["yyyymm","project","description","minute"]].groupby(["yyyymm","project","description"]).sum()

minute
yyyymm  project description                              
2021-03 AI      SIGNATE_Quest                          92
        English 英語耳                                    90
        Python  Create a batch that Links a and b     243
                Lemon_SIGNATE                         164
2021-04 AI      SIGNATE_Quest                         202
                物体検出入門                                 42
                食品ロス                                  121
        English 英語耳                                    55